# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1

Будем парсить страницу со свежеми новостям на https://habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [96]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [141]:
KEYWORDS = ['JSON', 'python', 'pact', 'UTM', 'DNS', 'HTML']
res = requests.get('https://habr.com/ru/all/')

In [142]:
soup = BeautifulSoup(res.text, 'html.parser')
soup

<!DOCTYPE html>

<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,viewport-fit=cover" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format('woff2');
      unicode-range: U+0400-045F, U+0490-0491, U+04B0-04B1, U+2116;
    }

    /* latin-ext */
    @font-face {
      

In [143]:
# из текста домашнего задания так и не понял с чем нужно работать на habr.com: со статьями или с новостями? Это все-таки разные 
# блоки на сайте. У новостей нет preview-информации, есть только заголовки. Но с другой стороны во время вебинара было сказано,
# что ДЗ "никак не связано с хабами, оно связано с текстами НОВОСТЕЙ". Решил, что буду производить поиск по статьям (не используя хабы)

posts = soup.find_all('article', class_='tm-articles-list__item') # получем список тегов со статьями 
posts

[<article class="tm-articles-list__item" data-navigatable="" id="573776" tabindex="0"><div class="tm-article-snippet"><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" href="/ru/users/wlkr/" title="wlkr"><div class="tm-entity-image"><svg class="tm-svg-img tm-image-placeholder tm-image-placeholder_blue" height="24" width="24"><!-- --><use xlink:href="/img/megazord-v24.cee85629.svg#placeholder-user"></use></svg></div></a><span class="tm-user-info__user"><a class="tm-user-info__username" href="/ru/users/wlkr/">
       wlkr
     </a></span></span><span class="tm-article-snippet__datetime-published"><time datetime="2021-08-19T15:07:46.000Z" title="2021-08-19, 18:07">сегодня в 18:07</time></span></div><!-- --></div><h2 class="tm-article-snippet__title tm-article-snippet__title_h2"><a class="tm-article-snippet__title-link" data-article-link="" href="/ru/post/573776/"><spa

In [146]:
articles = pd.DataFrame()
for post in posts:
    text = post.text.lower()
    for word in KEYWORDS:
        if word.lower() in text:
            date  = pd.to_datetime(post.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime')).strftime('%Y-%m-%d %H:%M:%S')
            title = post.find('a', class_='tm-article-snippet__title-link').text
            url = 'https://habr.com' + post.find('a', class_='tm-article-snippet__title-link').get('href')
            row = {'Дата': date, 'Заголовок': title, 'Ссылка': url}
            articles = pd.concat([articles, pd.DataFrame([row])], ignore_index=True)
# т.к. в какой-то статье может быть несколько ключевых слов, то в итоговом датафрейме возможно дублирование строк
# удаляем дубликаты
articles.drop_duplicates()

,Дата,Заголовок,Ссылка
0,2021-08-19 13:18:00,Как запустить Jupyter Notebook в браузере без ...,https://habr.com/ru/company/skillfactory/blog/...
1,2021-08-19 12:01:01,Как в восемь раз уменьшить количество DNS-запр...,https://habr.com/ru/company/ozontech/blog/570936/


## Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [147]:
# этот вариант находит чуть больше статей, т.к. ищет по всему тексту полной статьи
articles = pd.DataFrame()
for post in posts:
    # сначала получаем ссылку для каждой статьи
    url = 'https://habr.com' + post.find('a', class_='tm-article-snippet__title-link').get('href') 
    # делаем запрос по получившейся сссылке и находим тег, содержащий полную статью
    page = requests.get(url)
    full_article = BeautifulSoup(page.text, 'html.parser').find('article', class_='tm-page-article__content tm-page-article__content_inner')
    full_text = full_article.text.lower()
    for word in KEYWORDS:
        # если ключевое слово встречается в текстовом содержимом тега со статьей, то подбираем еще дату и заголовок статьи
        if word.lower() in full_text:
            date  = pd.to_datetime(post.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime')).strftime('%Y-%m-%d %H:%M:%S')
            title = post.find('a', class_='tm-article-snippet__title-link').text
            text = full_article.find('div', class_ = 'tm-article-body').text
            row = {'Дата': date, 'Заголовок': title, 'Ссылка': url, 'Текст статьи' : text}
            articles = pd.concat([articles, pd.DataFrame([row])], ignore_index=True)
# т.к. в какой-то статье может быть несколько ключевых слов, то в итоговом датафрейме возможно дублирование строк
# удаляем дубликаты   
articles.drop_duplicates()

,Дата,Заголовок,Ссылка,Текст статьи
0,2021-08-19 14:13:01,Молодые не идут в .NET. Правда или нет?,https://habr.com/ru/company/dododev/blog/573470/,Как-то раз наши разработчики тёрли за что-то в...
2,2021-08-19 13:55:31,Эффективный поиск XSS-уязвимостей,https://habr.com/ru/company/jugru/blog/569270/,\nПро XSS-уязвимости известно давным-давно — к...
4,2021-08-19 13:18:00,Как запустить Jupyter Notebook в браузере без ...,https://habr.com/ru/company/skillfactory/blog/...,К старту нашего флагманского курса по Data Sci...
5,2021-08-19 13:15:22,NLP At Scale: вся правда о предобученных модел...,https://habr.com/ru/company/mailru/blog/573718/,\r\nАнтиспам Почты Mail.ru — это симбиоз проду...
6,2021-08-19 12:49:11,Ускорение WordPress. Тотальный разбор плагинов...,https://habr.com/ru/post/573750/,В этой части я подробно рассмотрю принцип рабо...
8,2021-08-19 12:01:01,Как в восемь раз уменьшить количество DNS-запр...,https://habr.com/ru/company/ozontech/blog/570936/,"Привет, Хабр. Меня зовут Рустам. Я работаю в O..."
9,2021-08-19 11:54:04,"Как найти идеального кандидата, и что делать с...",https://habr.com/ru/company/agima/blog/573690/,"Поиск, подбор и найм новых сотрудников – практ..."


## Задание 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck(https://www.avast.com/hackcheck/). Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

In [64]:
EMAIL = ['kolya@mail.ru', 'tanya@gmail.com', 'vanya@yandex.ru', 'xxx@x.ru', 'yyy@y.com']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',

    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Version': '0',

    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/'
}
PAYLOAD = {'emailAddresses': EMAIL}
# Долго не получалось сделать POST запрос, выдавалась ошибка 400, хотя все атрибуты определил правильно. 
# В итоге честно взял этот кусок кода отсюда:
# https://ru.stackoverflow.com/questions/1242929/Как-загрузить-ответ-на-post-запрос-с-сайта-в-виде-json-файла

r = requests.post(URL, headers=HEADERS, json=PAYLOAD)
r

<Response [200]>

In [65]:
# для наглядности отобразим в формате JSON
r.json()

{'breaches': {'2': {'breachId': 2,
   'site': 'linkedin.com',
   'recordsCount': 158591429,
   'description': "In 2012, online professional networking platform LinkedIn fell victim to a breach of its members' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \n\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.",
   'publishDate': '2016-10-21T00:00:00Z',
   'statistics': {'usernames': 0,
    'passwords': 111975337,
    'emails': 158591429}},
  '3': {'breachId': 3,
   'site': 'adobe.com',
   'recordsCount': 152046506,
   'description': "In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software produc

In [66]:
# по заданию должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>
# не лишним будет также указывать и почту, для которой произошла утечка. У каждого события утечки есть свой номер - breachId
# сопоставим емэйл адрес и события утечки
email_id = pd.DataFrame()
for mail, breaches in r.json()['summary'].items():
    for breachId in breaches['breaches']:
#         print(breachId)
        row  = {'email': mail, 'breachId': int(breachId)}
        email_id = pd.concat([email_id, pd.DataFrame([row])], ignore_index=True)

email_id

,email,breachId
0,yyy@y.com,2
1,yyy@y.com,3
2,yyy@y.com,15
3,yyy@y.com,41
4,yyy@y.com,3520
...,...,...
112,tanya@gmail.com,37785
113,vanya@yandex.ru,16868
114,vanya@yandex.ru,17457
115,vanya@yandex.ru,18122


In [67]:
# подготовим датафрейм с подробными сведениями по каждой утечке данных
email_dframe = pd.DataFrame()
for breachId, event in r.json()['breaches'].items():
    row = {
        'breachId': int(breachId), 
        'Дата утечки': pd.to_datetime(event['publishDate']).date(), 
        'Источник утечки': event['site'], 
        'Описание утечки': event['description']}
    email_dframe = pd.concat([email_dframe, pd.DataFrame([row])], ignore_index=True)
email_dframe

,breachId,Дата утечки,Источник утечки,Описание утечки
0,2,2016-10-21,linkedin.com,"In 2012, online professional networking platfo..."
1,3,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
2,12,2016-10-29,vk.com,Popular Russian social networking platform VKo...
3,15,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
4,41,2016-10-24,dropbox.com,Cloud storage company Dropbox suffered a major...
...,...,...,...,...
94,37446,2021-04-29,bigbasket.com,"In October 2020, the Indian online grocery sto..."
95,37529,2021-05-27,shadowcraft.ru,"At an unknown date, the Russian Minecraft serv..."
96,37570,2021-05-27,dailyquiz.me,"In January 2021, the entertainment website Dai..."
97,37777,2021-07-15,unknown,"In June 2021, data from an unknown Russian tec..."


In [68]:
# получил разное количество строк в датафреймах email_dframe и email_id - это из-за того, что каждому событию утечки breachId
# можно соотнести несколько почтовых адресов. Чтобы убедиться, что количеситво уникальных breachId в обоих датафремах
# одинаково, удалю дублирующиеся
email_id.drop_duplicates(subset='breachId', ignore_index=True) 

,email,breachId
0,yyy@y.com,2
1,yyy@y.com,3
2,yyy@y.com,15
3,yyy@y.com,41
4,yyy@y.com,3520
...,...,...
94,tanya@gmail.com,37382
95,tanya@gmail.com,37446
96,tanya@gmail.com,37570
97,tanya@gmail.com,37785


In [69]:
# т.о. в обоих датафреймах имеется одинаковые множества breachId. Можно без потери данных объеденить датафреймы по этому столбцу
total_result = email_id.merge(email_dframe, how = 'outer', on = 'breachId')
#pd.set_option('display.max_rows', None)   - можно разкомментировать чтобы отключить ограничение на количество выводимых строк 
print('Итоговый датафрейм: ')
total_result

Итоговый датафрейм: 


,email,breachId,Дата утечки,Источник утечки,Описание утечки
0,yyy@y.com,2,2016-10-21,linkedin.com,"In 2012, online professional networking platfo..."
1,yyy@y.com,3,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
2,xxx@x.ru,3,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
3,yyy@y.com,15,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
4,xxx@x.ru,15,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
...,...,...,...,...,...
112,tanya@gmail.com,37382,2021-04-22,bookchor.com,"In February 2021, the online second-hand books..."
113,tanya@gmail.com,37446,2021-04-29,bigbasket.com,"In October 2020, the Indian online grocery sto..."
114,tanya@gmail.com,37570,2021-05-27,dailyquiz.me,"In January 2021, the entertainment website Dai..."
115,tanya@gmail.com,37785,2021-07-15,unknown,The proliferation of stolen or leaked-breach d...


## Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  

TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [121]:
# задаем параметры для запроса к методу wall.get
TOKEN = 'беспорядочныйнаборцифрибукв'
URL_REQUEST = 'https://api.vk.com/method/wall.get?'
GROUP = 'netology'
VERSION = '5.131'
params = {
    'domain': GROUP,
    'access_token': TOKEN,
    'v': VERSION,
    'count': 50
}

In [109]:
res = requests.get(URL_REQUEST, params) # Сам запрос

In [132]:
import datetime
D_frame=pd.DataFrame(res.json()['response']['items']) # Получаем полный датафрейм с сообщениями на стене сообщества
D_frame['date'] = pd.to_datetime(D_frame['date'], unit='s').dt.date # Преобразуем дату в человеческий вид
D_frame

,id,from_id,owner_id,date,marked_as_ads,post_type,text,is_pinned,attachments,post_source,comments,likes,reposts,views,is_favorite,donut,short_text_rate,hash,edited,carousel_offset
0,94620,-30159897,-30159897,2021-08-16,0,post,Наш летний диджитал-марафон продолжается 🎁\n \...,1.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 139, 'can_post': 1, 'groups_can_post...","{'count': 28, 'user_likes': 0, 'can_like': 1, ...","{'count': 2, 'user_reposted': 0}",{'count': 2824},False,{'is_donut': False},0.8,jJ426dvUtyATWNu2PMzRh7Tc,NaN,NaN
1,97232,-30159897,-30159897,2021-08-19,0,post,С появлением искусственного интеллекта тратить...,NaN,"[{'type': 'link', 'link': {'url': 'http://neto...",{'type': 'vk'},"{'count': 60, 'can_post': 1, 'groups_can_post'...","{'count': 18, 'user_likes': 0, 'can_like': 1, ...","{'count': 1, 'user_reposted': 0}",{'count': 1096},False,{'is_donut': False},0.8,kocLgoZTU3Bg5dVg0cg9aVAB,NaN,NaN
2,96837,-30159897,-30159897,2021-08-18,0,post,У нас сегодня Нето-викторина 🤩\n\nМы подготови...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'api'},"{'count': 393, 'can_post': 1, 'groups_can_post...","{'count': 15, 'user_likes': 0, 'can_like': 1, ...","{'count': 0, 'user_reposted': 0}",{'count': 1524},False,{'is_donut': False},0.8,t-GdKlq83ujBZpXeoRWGER4E,NaN,NaN
3,96438,-30159897,-30159897,2021-08-18,0,post,Запустили курс «Старт в SMM» 🙃\n \nSMM-специал...,NaN,"[{'type': 'link', 'link': {'url': 'http://neto...",{'type': 'vk'},"{'count': 327, 'can_post': 1, 'groups_can_post...","{'count': 29, 'user_likes': 0, 'can_like': 1, ...","{'count': 10, 'user_reposted': 0}",{'count': 33165},False,{'is_donut': False},0.8,r-5ioV1Fm_tT2RMcQV5ToK0y,1.629283e+09,NaN
4,95022,-30159897,-30159897,2021-08-17,0,post,Вебинар ничем не отличается от любого публично...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 791, 'can_post': 1, 'groups_can_post...","{'count': 51, 'user_likes': 0, 'can_like': 1, ...","{'count': 33, 'user_reposted': 0}",{'count': 13400},False,{'is_donut': False},0.8,LXkVQ9aeeh1tEaGy13UF4Pa_,NaN,0.0
5,94714,-30159897,-30159897,2021-08-17,0,post,💼 20 августа в 19:00 на нашем Ютуб-канале прой...,NaN,"[{'type': 'link', 'link': {'url': 'http://neto...",{'type': 'vk'},"{'count': 203, 'can_post': 1, 'groups_can_post...","{'count': 28, 'user_likes': 0, 'can_like': 1, ...","{'count': 1, 'user_reposted': 0}",{'count': 33614},False,{'is_donut': False},0.8,78TOgdpXGO5WEx6SjUt4wUQb,NaN,NaN
6,94662,-30159897,-30159897,2021-08-16,0,post,Коллаборации с блогерами — один из важных инст...,NaN,"[{'type': 'link', 'link': {'url': 'http://neto...",{'type': 'vk'},"{'count': 45, 'can_post': 1, 'groups_can_post'...","{'count': 31, 'user_likes': 0, 'can_like': 1, ...","{'count': 13, 'user_reposted': 0}",{'count': 16402},False,{'is_donut': False},0.8,-EIHOzPYevGGu55KDpA2IsCp,NaN,NaN
7,94610,-30159897,-30159897,2021-08-16,0,post,25 августа стартует бесплатный курс «Системный...,NaN,"[{'type': 'link', 'link': {'url': 'http://neto...",{'type': 'vk'},"{'count': 16, 'can_post': 1, 'groups_can_post'...","{'count': 28, 'user_likes': 0, 'can_like': 1, ...","{'count': 9, 'user_reposted': 0}",{'count': 72820},False,{'is_donut': False},0.8,pD3vdZEmZamC1EHJwq1sFzB2,1.629103e+09,NaN
8,94583,-30159897,-30159897,2021-08-15,0,post,Ещё немного и лето закончится 🙁 \n \nПредлагае...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 74, 'can_post': 1, 'groups_can_post'...","{'count': 74, 'user_likes': 0, 'can_like': 1, ...","{'count': 50, 'user_reposted': 0}",{'count': 23503},False,{'is_donut': False},0.8,H4qz_bzKRLXuFKVW0jNzahtj,NaN,NaN
9,94554,-30159897,-30159897,2021-08-14,0,post,Летний отпуск может серьёзно ударить по карман...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 156, 'can_post': 1, 'groups_can_post...","{'count': 37, 'user_likes': 0, 'can_like': 1, ...","{'count': 10, 'user_reposted': 0}",{'count': 60145},Fa

In [136]:
result_frame = D_frame.loc[:,['date','text']] # Оставляем только два нужных столбца
result_frame.columns = ['Дата поста', 'Текст поста'] # Переименовываем столбцы
result_frame

,Дата поста,Текст поста
0,2021-08-16,Наш летний диджитал-марафон продолжается 🎁\n \...
1,2021-08-19,С появлением искусственного интеллекта тратить...
2,2021-08-18,У нас сегодня Нето-викторина 🤩\n\nМы подготови...
3,2021-08-18,Запустили курс «Старт в SMM» 🙃\n \nSMM-специал...
4,2021-08-17,Вебинар ничем не отличается от любого публично...
5,2021-08-17,💼 20 августа в 19:00 на нашем Ютуб-канале прой...
6,2021-08-16,Коллаборации с блогерами — один из важных инст...
7,2021-08-16,25 августа стартует бесплатный курс «Системный...
8,2021-08-15,Ещё немного и лето закончится 🙁 \n \nПредлагае...
9,2021-08-14,Летний отпуск может серьёзно ударить по карман...
